In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook

In [2]:
def prepare_data(path, target):
    raw_df = pd.read_csv(path)
    
    site_names = ['site{}'.format(i) for i in range(1, 11)]
    time_names = ['time{}'.format(i) for i in range(1, 11)]
    feature_names = [None] * (2 * len(site_names))
    feature_names[1::2] = time_names
    feature_names[::2] = site_names
    
    # prepare 30-min steps
    raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'])
    time_diff = raw_df['timestamp'].diff().astype(int)
    time_diff = np.where(time_diff < 0, np.nan, time_diff)
    raw_df['min_diff'] = time_diff/(1e9*60)
    raw_df['min_diff'].fillna(0, inplace = True)
    raw_df['min_cumsum'] = raw_df['min_diff'].cumsum()
    raw_df['step'] = (raw_df['min_cumsum']//30).astype(int)
    
#     return raw_df
    
    step_list = raw_df['step'].unique()
    
    stacking_list = []
    for step in step_list:
        temp_part = raw_df[raw_df['step'] == step][['timestamp', 'site']].to_numpy()
        
#         print(temp_part.shape[0])
        
        # infill matrix by NaN`s
        if temp_part.shape[0]%10 != 0:
            temp_padding = np.full((10 - temp_part.shape[0]%10, 2), np.nan)
            temp_part = np.vstack([temp_part, temp_padding])

        # https://stackoverflow.com/questions/3678869/pythonic-way-to-combine-two-lists-in-an-alternating-fashion
        temp_combine = [None] * (2 * len(temp_part))
        temp_combine[1::2] = temp_part[:, 0]
        temp_combine[::2] = temp_part[:, 1]

        temp_result = np.array(temp_combine).reshape((-1, 20))
        stacking_list.append(temp_result)

    data = np.vstack(stacking_list)
    
    df = pd.DataFrame(data, columns = feature_names)
    df['target'] = np.full(df.shape[0], target)
    
    return df  

In [3]:
alice_path = 'Data/raw_train/Alice_log.csv'

In [4]:
%%time
alice_df = prepare_data(alice_path, target = 1)

CPU times: user 252 ms, sys: 4.3 ms, total: 256 ms
Wall time: 260 ms


In [5]:
alice_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,api.bing.com,2013-02-12 16:25:10,api.bing.com,2013-02-12 16:25:11,api.bing.com,2013-02-12 16:32:10,www.google.fr,2013-02-12 16:32:11,www.google.fr,2013-02-12 16:32:24,...,2013-02-12 16:32:25,www.google.fr,2013-02-12 16:32:25,www.info-jeunes.net,2013-02-12 16:32:26,platform.twitter.com,2013-02-12 16:32:27,www.info-jeunes.net,2013-02-12 16:32:27,1
1,www.facebook.com,2013-02-12 16:32:27,www.info-jeunes.net,2013-02-12 16:32:28,twitter.com,2013-02-12 16:32:29,www.info-jeunes.net,2013-02-12 16:32:34,www.info-jeunes.net,2013-02-12 16:32:35,...,2013-02-12 16:32:35,www.info-jeunes.net,2013-02-12 16:32:42,www.facebook.com,2013-02-12 16:32:42,www.info-jeunes.net,2013-02-12 16:32:51,www.info-jeunes.net,2013-02-12 16:32:53,1
2,www.facebook.com,2013-02-12 16:32:53,www.info-jeunes.net,2013-02-12 16:33:11,www.info-jeunes.net,2013-02-12 16:33:12,www.facebook.com,2013-02-12 16:33:13,twitter.com,2013-02-12 16:33:15,...,2013-02-12 16:33:24,www.facebook.com,2013-02-12 16:33:24,www.info-jeunes.net,2013-02-12 16:33:33,www.facebook.com,2013-02-12 16:33:34,api.bing.com,2013-02-12 16:33:46,1
3,www.bing.com,2013-02-12 16:33:50,www.bing.com,2013-02-12 16:33:51,www.leboncoin.fr,2013-02-12 16:33:52,www.bing.com,2013-02-12 16:33:52,twitter.com,2013-02-12 16:33:52,...,2013-02-12 16:33:52,deliv.leboncoin.fr,2013-02-12 16:33:52,www.leboncoin.fr,2013-02-12 16:33:53,deliv.leboncoin.fr,2013-02-12 16:33:53,static.leboncoin.fr,2013-02-12 16:33:53,1
4,193.164.197.30,2013-02-12 16:33:55,www.leboncoin.fr,2013-02-12 16:33:55,static.leboncoin.fr,2013-02-12 16:33:55,193.164.196.60,2013-02-12 16:33:55,193.164.197.40,2013-02-12 16:33:55,...,2013-02-12 16:33:55,193.164.196.40,2013-02-12 16:33:55,193.164.197.60,2013-02-12 16:33:55,193.164.197.50,2013-02-12 16:33:55,deliv.leboncoin.fr,2013-02-12 16:33:55,1


In [6]:
alice_df.shape

(2327, 21)

In [7]:
other_user_path = 'Data/raw_train/other_user_logs'

In [8]:
files_list = sorted([file for file in os.listdir(other_user_path) if 'csv' in file])

In [9]:
other_users_df = pd.DataFrame(columns = alice_df.columns)

for file_name in tqdm_notebook(files_list):
    temp_df = prepare_data(os.path.join(other_user_path, file_name), target = 0)
    other_users_df = pd.concat([other_users_df, temp_df])
    
other_users_df.reset_index(drop = True, inplace = True)
    

In [10]:
other_users_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,fpdownload2.macromedia.com,2013-11-29 08:14:18,hotmail.fr,2013-11-29 08:14:26,login.live.com,2013-11-29 08:14:38,login.live.com,2013-11-29 08:14:57,login.live.com,2013-11-29 08:15:17,...,2013-11-29 08:15:18,mail.live.com,2013-11-29 08:15:23,dub122.mail.live.com,2013-11-29 08:15:29,people.directory.live.com,2013-11-29 08:15:30,dub122.mail.live.com,2013-11-29 08:15:30,0
1,dub122.mail.live.com,2013-11-29 08:15:34,secure.shared.live.com,2013-11-29 08:15:35,windowslive.tt.omtrdc.net,2013-11-29 08:15:36,dub122.mail.live.com,2013-11-29 08:15:36,secure.shared.live.com,2013-11-29 08:15:37,...,2013-11-29 08:15:37,js.live.net,2013-11-29 08:15:37,people.directory.live.com,2013-11-29 08:15:37,login.live.com,2013-11-29 08:15:38,go.trouter.io,2013-11-29 08:15:39,0
2,storage.live.com,2013-11-29 08:15:39,blufiles.storage.msn.com,2013-11-29 08:15:39,h.live.com,2013-11-29 08:15:39,windowslive.tt.omtrdc.net,2013-11-29 08:15:39,prod.registrar.skype.com,2013-11-29 08:15:40,...,2013-11-29 08:15:41,h.live.com,2013-11-29 08:15:41,h.live.com,2013-11-29 08:15:42,secure.shared.live.com,2013-11-29 08:15:49,h.live.com,2013-11-29 08:16:47,0
3,h.live.com,2013-11-29 08:17:32,dub122.mail.live.com,2013-11-29 08:17:36,dub122.mail.live.com,2013-11-29 08:18:26,cid-1bed360223325286.users.storage.live.com,2013-11-29 08:18:31,proxy-bay-people.directory.live.com,2013-11-29 08:21:45,...,2013-11-29 08:24:57,proxy-bay-people.directory.live.com,2013-11-29 08:40:48,dub122.mail.live.com,2013-11-29 08:40:49,h.live.com,2013-11-29 08:40:50,NaN,NaT,0
4,proxy-bay-people.directory.live.com,2013-11-29 08:45:53,dub122.mail.live.com,2013-11-29 08:46:22,h.live.com,2013-11-29 08:47:26,dub122.mail.live.com,2013-11-29 08:47:27,proxy-bay-people.directory.live.com,2013-11-29 08:50:57,...,2013-11-29 08:59:53,prod.registrar.skype.com,2013-11-29 09:00:19,h.live.com,2013-11-29 09:00:57,prod.registrar.skype.com,2013-11-29 09:00:58,h.live.com,2013-11-29 09:01:09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261549,login.live.com,2014-03-17 16:58:12,login.live.com,2014-03-17 16:58:13,windowslive.tt.omtrdc.net,2014-03-17 16:58:15,login.live.com,2014-03-17 16:58:23,login.live.com,2014-03-17 16:58:24,...,2014-03-17 16:58:24,dub115.mail.live.com,2014-03-17 16:58:24,windowslive.tt.omtrdc.net,2014-03-17 16:58:24,dub115.mail.live.com,2014-03-17 16:58:26,people.directory.live.com,2014-03-17 16:58:27,0
261550,dub115.mail.live.com,2014-03-17 16:58:28,windowslive.tt.omtrdc.net,2014-03-17 16:58:28,cid-97d7198274048e15.users.storage.live.com,2014-03-17 16:58:28,h.live.com,2014-03-17 16:58:29,people.directory.live.com,2014-03-17 16:58:29,...,2014-03-17 16:58:29,js.live.net,2014-03-17 16:58:29,h.live.com,2014-03-17 16:58:30,secure.shared.live.com,2014-03-17 16:58:30,dub115.mail.live.com,2014-03-17 16:58:30,0
261551,dub115.mail.live.com,2014-03-17 16:58:31,h.live.com,2014-03-17 16:58:33,go.trouter.io,2014-03-17 16:58:33,storage.live.com,2014-03-17 16:58:47,secure.shared.live.com,2014-03-17 16:59:22,...,2014-03-17 16:59:25,p.sfx.ms,2014-03-17 16:59:25,cid-97d7198274048e15.users.storage.live.com,2014-03-17 16:59:30,h.live.com,2014-03-17 16:59:30,dub115.mail.live.com,2014-03-17 16:59:30,0
261552,go.trouter.io,2014-03-17 16:59:51,go.microsoft.com,2014-03-17 16:59:51,ieonline.microsoft.com,2014-03-17 16:59:51,windowslive.tt.omtrdc.net,2014-03-17 16:59:59,www.facebook.com,2014-03-17 17:00:07,...,2014-03-17 17:00:08,pbs.twimg.com,2014-03-17 17:00:08,eulerian.canal-plus.com,2014-03-17 17:00:08,pbs.twimg.com,2014-03-17 17:00:09,www.facebook.com,2014-03-17 17:00:09,0


In [11]:
other_users_df.shape

(261554, 21)

In [12]:
result_df = pd.concat([alice_df, other_users_df]).sort_values('time1')

result_df.reset_index(drop = True, inplace = True)
result_df['session_id'] = result_df.index.to_numpy() + 1

In [13]:
#reorder columns
columns = result_df.columns.to_list()
result_df = result_df[['session_id'] + columns[:-1]]
result_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,safebrowsing.clients.google.com,2013-01-12 08:05:57,safebrowsing-cache.google.com,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
1,2,safebrowsing.clients.google.com,2013-01-12 08:37:23,safebrowsing-cache.google.com,2013-01-12 08:37:23,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,3,www.apache.org,2013-01-12 08:50:13,www.apache.org,2013-01-12 08:50:14,download.eclipse.org,2013-01-12 08:50:15,www.apache.org,2013-01-12 08:50:15,www.apache.org,...,2013-01-12 08:50:16,download.oracle.com,2013-01-12 08:50:16,javadl-esd-secure.oracle.com,2013-01-12 08:50:16,www.caucho.com,2013-01-12 08:50:17,www.apache.org,2013-01-12 08:50:17,0
3,4,www.webtide.com,2013-01-12 08:50:17,download.oracle.com,2013-01-12 08:50:17,www.caucho.com,2013-01-12 08:50:18,download.oracle.com,2013-01-12 08:50:18,www.webtide.com,...,2013-01-12 08:50:18,public.dhe.ibm.com,2013-01-12 08:50:19,www.webtide.com,2013-01-12 08:50:19,www.apache.org,2013-01-12 08:50:19,www.apache.org,2013-01-12 08:50:20,0
4,5,public.dhe.ibm.com,2013-01-12 08:50:20,jope.ow2.org,2013-01-12 08:50:20,download.oracle.com,2013-01-12 08:50:20,public.dhe.ibm.com,2013-01-12 08:50:21,jope.ow2.org,...,2013-01-12 08:50:21,www.apache.org,2013-01-12 08:50:21,download.eclipse.org,2013-01-12 08:50:22,www.apache.org,2013-01-12 08:50:22,public.dhe.ibm.com,2013-01-12 08:50:22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263876,263877,ocsp.verisign.com,2014-04-30 23:33:48,ocsp.verisign.com,2014-04-30 23:33:49,ocsp.digicert.com,2014-04-30 23:33:52,gtssl-ocsp.geotrust.com,2014-04-30 23:33:52,ocsp.digicert.com,...,2014-04-30 23:33:53,clients1.google.com,2014-04-30 23:33:54,gtssl-ocsp.geotrust.com,2014-04-30 23:33:54,www.mozilla.org,2014-04-30 23:33:57,geo.mozilla.org,2014-04-30 23:34:00,0
263877,263878,dropbox.com,2014-04-30 23:34:15,ocsp.godaddy.com,2014-04-30 23:34:16,ocsp.godaddy.com,2014-04-30 23:34:17,clients1.google.com,2014-04-30 23:34:18,ocsp.verisign.com,...,2014-04-30 23:35:16,www.dropbox.com,2014-04-30 23:35:29,dl-web.dropbox.com,2014-04-30 23:36:12,www.dropbox.com,2014-04-30 23:36:42,ajax.googleapis.com,2014-04-30 23:37:13,0
263878,263879,clients1.google.com,2014-04-30 23:38:08,www.dropbox.com,2014-04-30 23:38:10,javadl-esd-secure.oracle.com,2014-04-30 23:38:13,javadl-esd-secure.oracle.com,2014-04-30 23:38:18,www.dropbox.com,...,2014-04-30 23:38:24,c.woopic.com,2014-04-30 23:38:35,orange.fr,2014-04-30 23:38:35,www.orange.fr,2014-04-30 23:38:35,c.orange.fr,2014-04-30 23:38:36,0
263879,263880,iapref.orange.fr,2014-04-30 23:38:36,c.woopic.com,2014-04-30 23:38:36,id.orange.fr,2014-04-30 23:38:38,r.orange.fr,2014-04-30 23:38:38,ocsp.godaddy.com,...,2014-04-30 23:38:40,id.orange.fr,2014-04-30 23:38:40,dl-web.dropbox.com,2014-04-30 23:39:07,dl-web.dropbox.com,2014-04-30 23:39:08,www.dropbox.com,2014-04-30 23:39:53,0


In [14]:
result_df.to_csv('Data/additional_train_data.csv')